## Part 1: Introduction

#### Project Background

In today's digital age, a Restaurant Recommender is an intelligent system designed to suggest restaurants based on user preferences, location, cuisine preferences, reviews, and other factors. Using machine learning and data analysis, it tailors recommendations to individual tastes, ensuring users discover new and highly-rated dining experiences. Whether you're searching for a hidden gem, a trendy spot, or a budget-friendly option, a Restaurant Recommender makes dining decisions effortless and personalized. To address this, we will develop **RestaurantRecommender AI, a chatbot that combines the power of large language models and rule-based functions to ensure accurate and reliable information delivery**.


#### Problem Statement

*Given Zomato restaurants dataset containing information about restaurants (restaurant names, restaurant types, rating, average cost, cuisine type, area etc.), I will build a chatbot that parses the dataset and provides accurate restaurants recommendations based on the user requirements*.

In [1485]:
# Install OpenAI library
!pip install -U -q openai tenacity

In [1486]:
# Import the libraries
import pandas as pd
from IPython.display import display, HTML
# Set the display width to control the output width
pd.set_option('display.width', 100)

# Read the dataset and read the Laptop Dataset
df = pd.read_csv('/Users/sahilavasthi/upGrad/genai/project/zomato.csv')
df.head(5)

,Unnamed: 0.1,Unnamed: 0,restaurant name,restaurant type,rate (out of 5),num of ratings,avg cost (two people),online_order,table booking,cuisines type,area,local address
0,0,0,#FeelTheROLL,Quick Bites,3.4,7,200.0,No,No,Fast Food,Bellandur,Bellandur
1,1,1,#L-81 Cafe,Quick Bites,3.9,48,400.0,Yes,No,"Fast Food, Beverages","Byresandra,Tavarekere,Madiwala",HSR
2,2,2,#refuel,Cafe,3.7,37,400.0,Yes,No,"Cafe, Beverages",Bannerghatta Road,Bannerghatta Road
3,3,3,'@ Biryani Central,Casual Dining,2.7,135,550.0,Yes,No,"Biryani, Mughlai, Chinese",Marathahalli,Marathahalli
4,4,4,'@ The Bbq,Casual Dining,2.8,40,700.0,Yes,No,"BBQ, Continental, North Indian, Chinese, Bever...",Bellandur,Bellandur


In [1487]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7105 entries, 0 to 7104
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0.1           7105 non-null   int64  
 1   Unnamed: 0             7105 non-null   int64  
 2   restaurant name        7105 non-null   object 
 3   restaurant type        7105 non-null   object 
 4   rate (out of 5)        7037 non-null   float64
 5   num of ratings         7105 non-null   int64  
 6   avg cost (two people)  7048 non-null   float64
 7   online_order           7105 non-null   object 
 8   table booking          7105 non-null   object 
 9   cuisines type          7105 non-null   object 
 10  area                   7105 non-null   object 
 11  local address          7105 non-null   object 
dtypes: float64(2), int64(3), object(7)
memory usage: 666.2+ KB


#### Approach:

1. **Conversation and Information Gathering**: The chatbot will utilize language models to understand and generate natural responses. Through a conversational flow, it will ask relevant questions to gather information about the user's requirements.
2. **Information Extraction**: Once the essential information is collected, rule-based functions come into play, extracting top 3 restaurant that best matches the user's needs. We will be using function calling to extract information from the database / csv file & data comparisons
3. **Personalized Recommendation**: Leveraging this extracted information, the chatbot engages in further dialogue with the user, efficiently addressing their queries and aiding them in finding the perfect laptop solution.

## Part 2: System Design

#### Dataset

We have a dataset `Zomato.csv` where  each row describes the features of a single restaurant and also has ratings, area, online order, cusine types, etc. at the end. The chatbot that we build will leverage LLMs to parse above described information in multiple columns and provide best top 3 restaurant recommendations.

The chatbot should ask a series of questions to the user,
- Determine the user's requirments. We will use 8 features to encapsulate the user's needs. The 8 features are as follows:
    - Restaurant type
    - Minimum rating
    - Minimum number of reviews
    - Average cost per person
    - Online order
    - Table booking
    - Area
    - Cuisines type 


- Confirm if the user's requirements have been correctly captured at the end.

After that the chatbot lists down the top 3 restaurants that are the most relevant, and engages in further conversation to help the user find the best one.

`Stage 1`

- Intent Clarity Layer

`Stage 2`

- Restaurant Mapping Layer
- Restaurant Information Extraction Layer

`Stage 3`

- Restaurant recommendation layer

##### Major functions behind the Chatbot

Let's now look at a brief overview of the major functions that form the chatbot. We'll take a deep dive later

- `initialize_conversation()`: This initializes the variable conversation with the system message.
- `get_chat_model_completions()`: This takes the ongoing conversation as the input and returns the response by the assistant
- `moderation_check()`: This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, it ends the conversation.
- `compare_restaurants_features_with_user_described_restaurant_features` - OpenAI function description for function calling API
- `get_restaurant_details()`: This function combines all the required columns from the dataset and create one additional column in the dict format for comparisons
-  `compare_restaurants_features_with_user_described_restaurant_features()`: This function takes the assistant's response and evaluates if the chatbot has captured the user's `profile clearly` in order to start comparing the restaurant features with user defined features and come back with the top 3 restaurant recommendations.
- `recommendation_validation()`: This function verifies that the restaurant recommendations are good enough, has score greater than equal to 5, and matches the user's requirements.
- `restaurant_recommendation()`: This function recommends the restaturants

## Part 3: Implementation

## Stage 1

### 3.1 - Import the libraries

Let's start by importing the libraries that we'll require for this project. Following are the ones:
- openai
- pandas
- os, json, ast

Make sure the api key is stored in the text file `OPENAI_API_Key.txt`.

In [1497]:
## Import the necessary libraries for building the project
import os,json,ast
import openai
import pandas as pd

from tenacity import retry, wait_random_exponential, stop_after_attempt

In [1498]:
filepath = '/Users/sahilavasthi/upGrad/genai/'

with open(filepath + "OpenAI_API_Key.txt", "r") as f:
  openai.api_key = ' '.join(f.readlines())

os.environ['OPENAI_API_KEY'] = openai.api_key.strip()

### 3.2 - Implementing / defining modular functions

Let's start with the first part of the implementation - building the `intent clarity` layer. As mentioned earlier, this layer helps in identifying the user requirements and passing it on to the product matching layer. Here are the functions that we would be using for building these layers:

- ### `initialize_conversation()`


In [1500]:
def initialize_conversation():
    delimiter = "####"

    system_message = f"""
    You are an intelligent restaurant recommnder expert and your goal is to find the best restaurant for a user.
    You are trying to understand the user's requirement for these restaurant's features: ('Restaurant type', 'Minimum rating', 'Minimum number of reviews', 'Average cost for 2 person', 'Online order'
    ,'Table booking', 'Area', 'Cuisines type').
    You need to ask relevant questions and understand the user need for each feature by analysing the user's responses.
    After understanding their requirements, you'll use a function call to suggest the top 3 restaurant with their respective user match score.
    Recommend these restaurants and answer any user's query about them.

    {delimiter} Here are certain guidelines that you need to follow:
    Don't ask questions about more than 2 features at a time.
    If the user's budget is less than says less than 40 INR, please mention that there are no restaurants in that range.
    Recommend the top3 restaurants in the following format:
    Start with a brief summary of each restaurant in the following format, in decreasing order of price of restaurants:
    1. <Restaurant Name> : <Major specifications of the restaurant>, The budget remaining after spending in the restaurant would be <Price in Rs>
    2. <Restaurant Name> : <Major specifications of the restaurant>, The budget remaining after spending in the restaurant would be <Price in Rs>
    3. <Restaurant Name> : <Major specifications of the restaurant>, The budget remaining after spending in the restaurant would be <Price in Rs>
    {delimiter}

    {delimiter}To find the top3 restaurants, you need to have the following chain of thoughts:
    Thought 1: Ask one question to understand the user's profile and requirements.
    If their primary reservation / online order  for the restaurant is unclear. Ask another question to comprehend their needs.
    Answer "Yes" or "No" to indicate if you understand the requirements.
    If yes, proceed to the next step. Otherwise, rephrase the question to capture their profile.

    Thought 2: Now, you are trying to understand the requirements for other features which you couldn't in the previous step.
    Ask questions to strengthen your understanding of the user's profile.
    Don't ask questions about more than 2 features at a time.
    Answer "Yes" or "No" to indicate if you understood all the needs of the features and are confident about the same.
    If yes, move to the next Thought. If no, ask question on the features whose needs you are unsure of.
    It is a good practice to ask question with a sound logic as opposed to directly citing the feature you want to understand the need for.{delimiter}
    {delimiter}

    {delimiter} Here is a sample conversation between the user and assistant:
    Assistant: "Hello! I'm here to help you find the perfect restaurant that suits your needs. Could you please share your requirements?"
    User: "Hi, I am looking for a restaurant which serves cuisine type atleast Biryani & Mughlai."
    Assistant: "Great! As you like Biryani & Mughlai, you likely want to order from a restaurant that serves these cuisines. Hence, the restaurant should be serving atleast these cuisines under the price and area of your interest. May I know what is the price and area do you primarily focus on? Understanding the specific type of restaurant work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "Yes, I am looking for an average price for 2 people around 1000 in the Marathahalli area."
    Assistant: "Thank you for providing that information. Do you have any number related to the rating of the restaurant from 1-5 and for the number of ratings? Understanding these would help me to futher narrow down the right list of restaurants for you."
    User: "Yes, rating should be atleast 3 and number of ratings should be atleast 50."
    Assistant: "Thank you for the information. To ensure I have a complete understanding of your needs, I have one more question: Are you looking for Casual Dining, Quick Bites, Pub, Takeaway, Delivery?"
    User: "Yes, I am looking for Casual Dining."
    Assistant: "Thank you for the information. Therefore, you are not looking for an online order and want to book the table in the restaturant. Could you kindly confirm this? This will help me to find the options that meet the specified requirements."
    User: "Yes, I am looking for a restaurant where I can book a table."
    Assistant: "Get the answers from the User for all the seven parameters which are restaurant type, minimum rating, minimum number of reviews, average cost for 2 person, online order, table booking, area and cuisines type"
    Assistant: "Thank you for confirming. Based on your requirements, I will now find the top 3 restaurants that match your criteria. Please hold on for a moment."
    Assistant: "Explicity states which of the requirements did not match user requirements:"
    Assistant: "Start with a brief summary of each restaturant in the following format, in decreasing order of price of restaturants:"
    Assistant: "Here are the top 3 restaurants that match your criteria:"
    1. <Restaurant Name> : <Major specifications of the restaurant>, The budget remaining after spending in the restaurant would be <Price in Rs>
    2. <Restaurant Name> : <Major specifications of the restaurant>, The budget remaining after spending in the restaurant would be <Price in Rs>
    3. <Restaurant Name> : <Major specifications of the restaurant>, The budget remaining after spending in the restaurant would be <Price in Rs>
    {delimiter}

    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message }]
    return conversation

## Function Description for the Function Calling API

1. ### OpenAI tools call `compare_restaurants_features_with_user_described_restaurant_features()`: This function takes the assistant's response and evaluates if the chatbot has captured the user's profile

In [1502]:
function_descriptions = [
            {
                "name": "compare_restaurants_with_user_described_features",
                "description": "Get the top 3 restaurants from the catalogue, that best matches what the user is asking based on 'restaurant type','minimum rating','minimum number of reviews',"
                "'average cost for 2 person', 'online order' , 'table booking', 'area' and 'cuisines type'.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "restaurant type": {
                            "type": "string",
                            "description": "The requirement of the user in restaurant type classfied" ,
                        },
                        "minimum rating": {
                            "type": "string",
                            "description": "The minimum rating of the restaurant as per the user requirement" ,
                        },
                        "average cost for 2 person": {
                            "type": "string",
                            "description": "The average cost for 2 person of the restaurant as per the user requirement" ,
                        },
                        "cuisines type": {
                            "type": "string",
                            "description": "The requirement of the user in cuisine type classfied" ,
                        },
                        "area": {
                            "type": "string",
                            "description": "The area of the restaurant as per the user requirement" ,
                        },
                        "online order": {
                            "type": "string",
                            "description": "The requirement of the user for restaurant's online order classfied as yes or no" ,
                        },
                        "table booking": {
                            "type": "string",
                            "description": "The requirement of the user for restaurant's table booking classfied as yes or no" ,
                        },
                        "minimum number of reviews": {
                            "type": "integer",
                            "description": "The minimum number of reviews of the restaurant as per the user requirement" ,
                        },

                    },
                    "required": ["restaurant type","minimum rating","average cost for 2 person","cuisines type","area","online order","table booking","minimum number of reviews"],
                },
            }
        ]

Let's now look at the next functions.

- ### `get_chat_model_completions()`: This takes the ongoing conversation as the input and returns the response by the assistant. We'll use the Chat Completions function for performing LLM calls to OpenAI.

In [1504]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_model_completions(messages):
    response = openai.chat.completions.create(
        model="gpt-4-0613",
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output. This is set to 0 to get deterministic output.
        max_tokens = 500,

        functions=function_descriptions,
        function_call="auto",
    )
    return response.choices[0].message

- ### `moderation_check()`:
 This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, you can add a break statement to end the conversation.

In [1506]:
# Define a function called moderation_check that takes user_input as a parameter.

def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai.moderations.create(input=user_input, model="omni-moderation-latest")

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if moderation_output == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

## Stage 2 - Implementing the Product Mapping and Information Extraction Layers

- ### `product_map_layer()`:
 This function combines all the required columns from the dataset file and create one additional column in the dict format for comparisons.

In [1509]:
# Implement the function to get the restaurant details

def get_restaurant_details():
# Read the dataset
        df = pd.read_csv('/Users/sahilavasthi/upGrad/genai/project/' + 'zomato.csv')
        # Create a new column 'restaurant_details' in the DataFrame, which will contain a dictionary of restaurant details.
        df['restaurant_details'] = df.apply(lambda x: dict({
                'restaurant type': x['restaurant type'],
                'area': x['area'],
                'cuisines type': x['cuisines type'],
                'average cost for 2 person': x['avg cost (two people)'],
                'minimum number of reviews': x['num of ratings'],
                'minimum rating': x['rate (out of 5)'],
                'online order': x['online_order'],
                'table booking': x['table booking']
        }), axis=1)

        df_with_restaurant_details = df.copy()
        return df_with_restaurant_details
        

#type(df['restaurant_details'][2].get('cuisines type'))


- ### `compare_restaurants_with_user_described_features()`:
 This function is used to compare the user defined requirements with the restaturant dataset

In [1511]:
def compare_restaurants_with_user_described_features(user_req_string):
  # Read the dataset, and create a new column 'restaurant_details' in the DataFrame, which will contain a dictionary of restaurant details.  
  df_with_restaurant_details = get_restaurant_details()

  # Create 'Score' column in the DataFrame and initialize to 0
  df_with_restaurant_details['Score'] = 0
  for index, row in df_with_restaurant_details.iterrows():
      user_product_match_str = row['restaurant_details']
      score = 0
      budget_remaining = 0.0

      for key, user_value in user_req_string.items():
        #if key.lower() == 'average cost for 2 person':
        #    continue  # Skip budget comparison
        restaurant_value = user_product_match_str.get(key, None)

        if key == 'area':
            if user_value.lower() == restaurant_value.lower():
                ### If the area value is equal to the user value the score is incremented by 1
                score += 1
            else:
                break

        if key == 'restaurant type':
              if user_value.lower() in restaurant_value.lower():
                 ### If the restaurant type value is equal to the user value the score is incremented by 1
                 score += 1
              else:
                break
              
        if key == 'online order':
            if user_value.lower() == restaurant_value.lower():
                ### If the online order value is equal to the user value the score is incremented by 1
                score += 1
            else:
                continue

        if key == 'table booking':
            if user_value.lower() == restaurant_value.lower():
                ### If the table booking value is equal to the user value the score is incremented by 1
                score += 1
            else:
                continue

        if key == 'minimum number of reviews':
            ### If the minimum number of reviews is equal to the user value the score is incremented by 1
            if restaurant_value >= user_value:
                score += 1
            if restaurant_value < user_value:
                continue
            
        if key == 'minimum rating':
            ### If the minimum rating is equal to the user value the score is incremented by 1
            if restaurant_value >= float(user_value):
                score += 1
            if restaurant_value < float(user_value):
                break

        if key == 'average cost for 2 person':
            ### If the minimum rating is equal to the user value the score is incremented by 1
            if restaurant_value <= float(user_value):
                score += 1
                budget_remaining = float(user_value) - restaurant_value
            if restaurant_value > float(user_value):
                break

        if key == 'cuisines type':
           if user_value.lower() in restaurant_value.lower():
                ### If the cuisines value is equal to the user value the score is incremented by 1
                score += 1
                if len(restaurant_value.lower().split(',')) > len(user_value.lower().split(',')):
                    ### If the cuisines value is equal to the user value the score is incremented by 1
                    score += 1
           else:
               break

              
      # Update the score in the DataFrame          
      df_with_restaurant_details.loc[index, 'Score'] = score
      df_with_restaurant_details.loc[index, 'Budget_remaining'] = budget_remaining

  # Sort the restaurant by score in descending order and return the top 3 products
  df_with_restaurant_details.to_csv("/Users/sahilavasthi/upGrad/genai/shop-assist/updated_reastaurant.csv",index=False,header = True) 
  top_restaurants = df_with_restaurant_details.drop('restaurant_details', axis=1)
  top_restaurants = top_restaurants.sort_values(by=['Score', 'Budget_remaining', 'rate (out of 5)'], ascending=[False, False, False]).head(3)
  #top_restaurants.head()  


  return top_restaurants.to_json(orient='records')

- ### `product_validation_layer()`:

This function verifies that the restaurant recommendations are good enough, has score greater than equal to 5, and matches the user's requirements.

In [1513]:
def recommendation_validation(restaurant_recommendation):
    data = json.loads(restaurant_recommendation)
    data1 = []
    for i in range(len(data)):
      if data[i]['Score'] >= 5:
        data1.append(data[i])

    return json.dumps(data1)

## Stage 3 - Relevant Restaurants are recommended to the user

In [1515]:
def restaurant_recommendation(conversation):
    recommendation = get_chat_model_completions(conversation)
    moderation = moderation_check(recommendation.content.strip())
    if moderation == 'Flagged':
        #display("Sorry, this message has been flagged. Please restart your conversation.")
        return 'Flagged'
    conversation.append({"role": "assistant", "content": recommendation.content})
    print("\n" +recommendation.content + "\n")

In [1516]:
def dialogue_mgmt_system():
    conversation = initialize_conversation()
    introduction = get_chat_model_completions(conversation).content
    print(introduction + '\n')
    user_input = ""
    while(user_input != "exit"):
      user_input = input("")
        
      moderation = moderation_check(user_input)
      if moderation == 'Flagged':
          display("Sorry, this message has been flagged. Please restart your conversation.")
          break
            
      conversation.append({"role": "user", "content": user_input})
        
      # Step 1: Pass the user input to GPT
      response_assistant = get_chat_model_completions(conversation)
      if response_assistant.content != None:
          moderation = moderation_check(response_assistant.content.strip())
          if moderation == 'Flagged':
              display("Sorry, this message has been flagged. Please restart your conversation.")
              break
    
      if response_assistant.function_call:
        print("\nThank you for providing all the information. Kindly wait, while I fetch the products\n")
          
        # Step 2: Extract top3 restaurants by calling the external function
        function_name = response_assistant.function_call.name
        function_args = json.loads(response_assistant.function_call.arguments)
        print(f"User requirement: {function_args}")
        top_3_restaurants = compare_restaurants_with_user_described_features(function_args)
        function_response = recommendation_validation(top_3_restaurants)
        if len(function_response) == 0:
          print("Sorry, we do not have restaurants that match your requirements. Connecting you to a human expert.")
          break
            
        # Step 3: send the info on the function call and function response to GPT
        conversation.append(response_assistant)
        conversation.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )
        moderation = restaurant_recommendation(conversation)
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break
      else:
        conversation.append({"role": "assistant", "content": response_assistant.content})
        print("\n" +  response_assistant.content + "\n")

In [1517]:
dialogue_mgmt_system()

Assistant: "Hello! I'm here to help you find the perfect restaurant that suits your needs. Could you please share your requirements?"



  I am looking for restaurant with budget of 5000



That's a good start. Could you please specify the type of restaurant you are looking for? For example, are you interested in Casual Dining, Quick Bites, Pub, Takeaway, or Delivery? Also, could you let me know the type of cuisine you prefer?



 Quick Bites and cuisine will be North Indian



Great! You prefer Quick Bites and North Indian cuisine. Now, could you please tell me your preferred area? Also, do you have any specific requirements for the restaurant's rating and the minimum number of reviews it should have?



...


Thank you for providing all the information. Kindly wait, while I fetch the products

User requirement: {'restaurant type': 'Quick Bites', 'minimum rating': '2', 'average cost for 2 person': '5000', 'cuisines type': 'North Indian', 'area': 'Malleshwaram', 'online order': 'No', 'table booking': 'Yes', 'minimum number of reviews': 20}

Here are the top 3 restaurants that match your criteria:

1. New Krishna Bhavan: This Quick Bites type restaurant serves South Indian, North Indian, and Chinese cuisines. It has a rating of 4.2 with 600 reviews. However, it does not offer online ordering or table booking. The average cost for two people is 200 INR. The budget remaining after spending in the restaurant would be 4800 INR.

2. Dhanbad Food Stall: This Quick Bites type restaurant serves Bihari, North Indian, and Chinese cuisines. It has a rating of 2.8 with 40 reviews. It does not offer online ordering or table booking. The average cost for two people is 200 INR. The budget remaining after sp

 change area to Old Airport Road



Thank you for providing all the information. Kindly wait, while I fetch the products

User requirement: {'restaurant type': 'Quick Bites', 'minimum rating': '2', 'average cost for 2 person': '5000', 'cuisines type': 'North Indian', 'area': 'Old Airport Road', 'online order': 'No', 'table booking': 'Yes', 'minimum number of reviews': 20}

Here are the top 3 restaurants that match your criteria in the Old Airport Road area:

1. Cafe Srinidhi: This Quick Bites type restaurant serves South Indian, North Indian, Fast Food, and Street Food. It has a rating of 3.1 with 91 reviews. However, it does not offer online ordering or table booking. The average cost for two people is 200 INR. The budget remaining after spending in the restaurant would be 4800 INR.

2. Dine @ 9 Paratha Corner: This Quick Bites type restaurant serves North Indian and Chinese cuisines. It has a rating of 3.6 with 33 reviews. It does not offer online ordering or table booking. The average cost for two people is 300 INR. 

 exit



Sure, if you have any more questions in the future, feel free to ask. Have a great day!



In [1518]:
exit



In [1536]:
dialogue_mgmt_system()

Assistant: "Hello! I'm here to help you find the perfect restaurant that suits your needs. Could you please share your requirements?"



 I am looking for restaurant with budget of 500



Sure, I can help you with that. To narrow down the options, could you please specify the type of restaurant you are looking for? For example, are you interested in Casual Dining, Quick Bites, Pub, Takeaway, or Delivery? Also, could you please tell me your preferred area?



 Quick Bites and area would be Malleshwaram



Great! Now, could you please tell me your preferences regarding the cuisine type? Also, do you have any specific requirements for the minimum rating of the restaurant?



...


Thank you for your response. So, you are looking for a Quick Bites type restaurant in the Malleshwaram area that serves Seafood, has a minimum rating of 3, and offers online ordering. Do you have any specific requirements for the minimum number of reviews the restaurant should have?



 reviews could be 50



Thank you for providing all the information. Kindly wait, while I fetch the products

User requirement: {'restaurant type': 'Quick Bites', 'minimum rating': '3', 'average cost for 2 person': '500', 'cuisines type': 'Seafood', 'area': 'Malleshwaram', 'online order': 'Yes', 'table booking': 'No', 'minimum number of reviews': 50}

Here are the top 3 restaurants that match your criteria:

1. Halli Mane Donne Biriyani: This Quick Bites type restaurant is located in Rajajinagar, Malleshwaram. It serves Biryani and Seafood, has a rating of 4.1 with 87 ratings, and offers online ordering. The average cost for two people is 400 INR. The budget remaining after spending in the restaurant would be 100 INR.

2. Fish Corner: This Quick Bites type restaurant is located in Yeshwantpur, Malleshwaram. It serves North Indian, Chinese, and Seafood, has a rating of 3.7 with 118 ratings, and offers online ordering. The average cost for two people is 400 INR. The budget remaining after spending in the resta

 ok Can I now buy a Gun


'Sorry, this message has been flagged. Please restart your conversation.'

In [1538]:
dialogue_mgmt_system()

Assistant: "Hello! I'm here to help you find the perfect restaurant that suits your needs. Could you please share your requirements?"



 I am looking for a restaurant type for Delivery



Great! You're looking for a restaurant that offers delivery. To help me find the best options for you, could you please tell me your preferred cuisine type and the area where you want the delivery?



 Bakery and area would be Indiranagar



Perfect! You're looking for a bakery that delivers in Indiranagar. To further narrow down the options, could you please tell me your budget for 2 people and your minimum rating for the restaurant?



...


Thank you for sharing your budget and rating preference. Now, could you please tell me if you have a preference for the minimum number of reviews a restaurant should have? Also, do you prefer restaurants that offer online ordering and table booking options?



 reviews may be 50 I will prefer table booking



Thank you for providing all the information. Kindly wait, while I fetch the products

User requirement: {'restaurant type': 'Bakery', 'minimum rating': '2', 'average cost for 2 person': '1000', 'cuisines type': 'Bakery', 'area': 'Indiranagar', 'online order': 'No', 'table booking': 'Yes', 'minimum number of reviews': 50}

Here are the top 3 restaurants that match your criteria:

1. Lazy Suzy: This is a Cafe and Bakery type restaurant located in Indiranagar. It has a rating of 4.3 with 1235 reviews. The average cost for two people is 1000. They offer online ordering and table booking. The budget remaining after spending in the restaurant would be 0 INR.

2. The Rolling Pin: This is a Bakery and Dessert Parlor located in Indiranagar. It has a rating of 4.0 with 74 reviews. The average cost for two people is 400. They do not offer online ordering and table booking. The budget remaining after spending in the restaurant would be 600 INR.

3. KrumbKraft: This is a Bakery located in Indirana

 exit



Sure, feel free to reach out anytime if you need more help. Have a great day!

